## Following Eye to Heatmap

In [1]:
import cv2
import mediapipe as mp
import numpy as np

In [12]:
import cv2
import mediapipe as mp
import numpy as np

def add_gaussian_to_heatmap(heatmap, center, sigma, amplitude):
    """
    Adds a Gaussian patch to the heatmap at the specified location.

    Parameters:
        heatmap (numpy.ndarray): Activation matrix (h x w) representing gaze distribution.
        center (tuple): Coordinates (cx, cy) of the Gaussian center.
        sigma (int): Standard deviation of the Gaussian distribution.
        amplitude (int): Maximum intensity value to add.

    Returns:
        numpy.ndarray: Updated heatmap with the added Gaussian patch.
    """
    h, w = heatmap.shape
    y, x = np.indices((h, w))
    cx, cy = center

    gaussian = amplitude * np.exp(-((x - cx)**2 + (y - cy)**2) / (2 * sigma**2))
    heatmap += gaussian
    return heatmap

def nothing(x):
    pass

# Create window and sliders for interactive parameter tuning
cv2.namedWindow("Gaze Heatmap")
cv2.createTrackbar("Decay", "Gaze Heatmap", 98, 100, nothing)       # 0.98 by default
cv2.createTrackbar("Sigma", "Gaze Heatmap", 15, 50, nothing)        # 15 by default
cv2.createTrackbar("Amplitude", "Gaze Heatmap", 50, 255, nothing)   # 50 by default

# Initialize MediaPipe Face Mesh with iris refinement
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
    refine_landmarks=True
)

# Initialize video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

ret, frame = cap.read()
h, w, _ = frame.shape
heatmap_gaze = np.zeros((h, w), dtype=np.float32)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Read slider values for real-time control
    decay_factor = cv2.getTrackbarPos("Decay", "Gaze Heatmap") / 100.0
    sigma = cv2.getTrackbarPos("Sigma", "Gaze Heatmap")
    amplitude = cv2.getTrackbarPos("Amplitude", "Gaze Heatmap")

    heatmap_gaze *= decay_factor

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Use iris landmarks for more accurate gaze point
            iris_left = [face_landmarks.landmark[i] for i in range(468, 472)]
            iris_right = [face_landmarks.landmark[i] for i in range(473, 477)]

            iris_left_px = [(int(p.x * w), int(p.y * h)) for p in iris_left]
            iris_right_px = [(int(p.x * w), int(p.y * h)) for p in iris_right]

            center_left_iris = (
                int(sum(p[0] for p in iris_left_px) / len(iris_left_px)),
                int(sum(p[1] for p in iris_left_px) / len(iris_left_px))
            )
            center_right_iris = (
                int(sum(p[0] for p in iris_right_px) / len(iris_right_px)),
                int(sum(p[1] for p in iris_right_px) / len(iris_right_px))
            )

            # Estimate gaze point as the average of both iris centers
            gaze_point = (
                (center_left_iris[0] + center_right_iris[0]) // 2,
                (center_left_iris[1] + center_right_iris[1]) // 2
            )

            # Add Gaussian patch to gaze heatmap
            heatmap_gaze = add_gaussian_to_heatmap(heatmap_gaze, gaze_point, sigma, amplitude)

    # Normalize and colorize heatmap for visualization
    heatmap_vis = cv2.normalize(heatmap_gaze, None, 0, 255, cv2.NORM_MINMAX)
    heatmap_vis = np.uint8(heatmap_vis)
    colored_heatmap = cv2.applyColorMap(heatmap_vis, cv2.COLORMAP_JET)

    # Overlay heatmap on the original frame
    overlay = cv2.addWeighted(frame, 0.6, colored_heatmap, 0.4, 0)

    cv2.imshow("Gaze Heatmap", overlay)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Save the final heatmap image
cv2.imwrite("final_gaze_heatmap.png", colored_heatmap)

# Cleanup
cap.release()
cv2.destroyAllWindows()
face_mesh.close()


## Following Eyes Heatmap in Video Recorded

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt

In [10]:
video_path = "Video Useful\head-pose-face-detection-male.mp4"

<>:1: SyntaxWarning: invalid escape sequence '\h'
<>:1: SyntaxWarning: invalid escape sequence '\h'
C:\Users\AlexisBenitez\AppData\Local\Temp\ipykernel_27820\173540399.py:1: SyntaxWarning: invalid escape sequence '\h'
  video_path = "Video Useful\head-pose-face-detection-male.mp4"


In [ ]:
def add_gaussian_to_heatmap(heatmap, center, sigma, amplitude):
    """
    Adds a Gaussian patch to the heatmap at the specified 'center'.
    
    Parameters:
        heatmap (numpy.ndarray): Activation matrix (h x w).
        center (tuple): (cx, cy) center of the patch.
        sigma (int): Standard deviation of the Gaussian.
        amplitude (int): Maximum value to be added.
    
    Returns:
        numpy.ndarray: Updated heatmap.
    """
    h, w = heatmap.shape
    y, x = np.indices((h, w))
    cx, cy = center
    gaussian = amplitude * np.exp(-((x - cx)**2 + (y - cy)**2) / (2 * sigma**2))
    heatmap += gaussian
    return heatmap

# Parameters
video_path = "Video Useful\head-pose-face-detection-male.mp4"            # Path to input video
output_video_path = "Save Images & Videos/output_pose_estimation.mp4"  # Output video file
decay_factor = 0.98                       # Heatmap decay factor (between 0 and 1)
sigma = 15                                # Standard deviation for Gaussian patch
amplitude = 50                            # Amplitude of the Gaussian patch

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5,
                                  min_tracking_confidence=0.5)

# Open input video and prepare VideoWriter
cap = cv2.VideoCapture(video_path)

ret, frame = cap.read()
h, w, _ = frame.shape # Get dimension from frame 

# Initialize gaze heatmap
heatmap_gaze = np.zeros((h, w), dtype=np.float32)

# Save the output file with mp4 format
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(output_video_path, fourcc, fps, (w, h))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Apply decay to fade old activations
    heatmap_gaze *= decay_factor

    # Convert frame to RGB for processing with MediaPipe
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Typical eye landmarks: 33 (left eye) and 263 (right eye)
            left_eye = face_landmarks.landmark[33]
            right_eye = face_landmarks.landmark[263]
            left_eye_coords = (int(left_eye.x * w), int(left_eye.y * h))
            right_eye_coords = (int(right_eye.x * w), int(right_eye.y * h))

            # Draw circles on the eyes (optional)
            cv2.circle(frame, left_eye_coords, 3, (0, 255, 0), -1)
            cv2.circle(frame, right_eye_coords, 3, (0, 255, 0), -1)

            # Approximate midpoint between eyes (proxy for gaze direction)
            mid_eye = (int((left_eye_coords[0] + right_eye_coords[0]) / 2),
                       int((left_eye_coords[1] + right_eye_coords[1]) / 2))

            # Add a Gaussian patch to the heatmap at the mid_eye location
            heatmap_gaze = add_gaussian_to_heatmap(heatmap_gaze, mid_eye, sigma, amplitude)

    # Normalize and create a colorized heatmap visualization
    heatmap_vis = cv2.normalize(heatmap_gaze, None, 0, 255, cv2.NORM_MINMAX)
    heatmap_vis = np.uint8(heatmap_vis)
    colored_heatmap = cv2.applyColorMap(heatmap_vis, cv2.COLORMAP_JET)

    # Overlay heatmap on the original frame
    overlay = cv2.addWeighted(frame, 0.6, colored_heatmap, 0.4, 0)

    # Write processed frame to the output video
    writer.write(overlay)

# Release resources
cap.release()
writer.release()
face_mesh.close()

print(f"Processed video saved at: {output_video_path}")


<>:22: SyntaxWarning: invalid escape sequence '\h'
<>:22: SyntaxWarning: invalid escape sequence '\h'
C:\Users\AlexisBenitez\AppData\Local\Temp\ipykernel_27820\286615450.py:22: SyntaxWarning: invalid escape sequence '\h'
  video_path = "Video Useful\head-pose-face-detection-male.mp4"            # Path to input video


Processed video saved at: Video Useful/output_pose_estimation.mp4
